In [1]:
!git clone 'https://github.com/aakashvardhan/s6-backprop.git'

Cloning into 's6-backprop'...
remote: Enumerating objects: 217, done.
remote: Counting objects: 100% (217/217), done.
remote: Compressing objects: 100% (146/146), done.
remote: Total 217 (delta 134), reused 148 (delta 68), pack-reused 0
Receiving objects: 100% (217/217), 652.24 KiB | 5.88 MiB/s, done.
Resolving deltas: 100% (134/134), done.


In [2]:
import sys
sys.path.append('/content/s6-backprop/Part 2')

In [3]:
from model import Net, model_summary
import torch
!pip install torchsummary

In [4]:
# Check if CUDA is available on the system and set `use_cuda` accordingly
use_cuda = torch.cuda.is_available()

# Set the device to "cuda" if CUDA is available, otherwise fall back to using the CPU
device = torch.device("cuda" if use_cuda else "cpu")

# Initialize the model and move it to the specified device (either GPU or CPU)
model = Net().to(device)

In [5]:
model_summary(model)

RuntimeError: Expected 2D (unbatched) or 3D (batched) input to conv1d, but got input of size: [2, 32, 14, 14]